# Graph Classification

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd
import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

In [6]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from dataclasses import dataclass
from typing import Optional
import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
import torch
import torch.nn as nn
import numpy as np
from typing import Union, Optional, Callable
from torch_scatter import scatter_add, scatter_max
from torch_geometric.utils import softmax
import math
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GCNConv, GATConv, ChebConv, GraphConv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.data import Data
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import softmax, dense_to_sparse, add_remaining_self_loops
from torch_scatter import scatter_add
from torch_sparse import spspmm, coalesce
import torch
import torch.nn as nn
from torch.autograd import Function
from torch_scatter import scatter_add, scatter_max
def topk(x, ratio, batch, min_score=None, tol=1e-7):
    if min_score is not None:
        scores_max = scatter_max(x, batch)[0][batch] - tol
        scores_min = scores_max.clamp(max=min_score)
        perm = torch.nonzero(x > scores_min).view(-1)
    else:
        num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
        batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()
        cum_num_nodes = torch.cat(
            [num_nodes.new_zeros(1),
            num_nodes.cumsum(dim=0)[:-1]], dim=0)
        index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
        index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
        dense_x = x.new_full((batch_size * max_num_nodes, ), -2)
        dense_x[index] = x
        dense_x = dense_x.view(batch_size, max_num_nodes)
        _, perm = dense_x.sort(dim=-1, descending=True)
        perm = perm + cum_num_nodes.view(-1, 1)
        perm = perm.view(-1)
        k = (ratio * num_nodes.to(torch.float)).ceil().to(torch.long)
        mask = [
            torch.arange(k[i], dtype=torch.long, device=x.device) +
            i * max_num_nodes for i in range(batch_size)
        ]
        mask = torch.cat(mask, dim=0)
        perm = perm[mask]
    return perm
def filter_adj(edge_index, edge_weight, perm, num_nodes=None):
        num_nodes = maybe_num_nodes(edge_index, num_nodes)
        mask = perm.new_full((num_nodes, ), -1)
        i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
        mask[perm] = i
        row, col = edge_index
        row, col = mask[row], mask[col]
        mask = (row >= 0) & (col >= 0)
        row, col = row[mask], col[mask]
        if edge_weight is not None:
            edge_weight = edge_weight[mask]
        return torch.stack([row, col], dim=0), edge_weight
def scatter_sort(x, batch, fill_value=-1e16):
    num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
    batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()
    cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)
    index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
    index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
    dense_x = x.new_full((batch_size * max_num_nodes,), fill_value)
    dense_x[index] = x
    dense_x = dense_x.view(batch_size, max_num_nodes)
    sorted_x, _ = dense_x.sort(dim=-1, descending=True)
    cumsum_sorted_x = sorted_x.cumsum(dim=-1)
    cumsum_sorted_x = cumsum_sorted_x.view(-1)
    sorted_x = sorted_x.view(-1)
    filled_index = sorted_x != fill_value
    sorted_x = sorted_x[filled_index]
    cumsum_sorted_x = cumsum_sorted_x[filled_index]
    return sorted_x, cumsum_sorted_x
def _make_ix_like(batch):
    num_nodes = scatter_add(batch.new_ones(batch.size(0)), batch, dim=0)
    idx = [torch.arange(1, i + 1, dtype=torch.long, device=batch.device) for i in num_nodes]
    idx = torch.cat(idx, dim=0)
    return idx
def _threshold_and_support(x, batch):
    """Sparsemax building block: compute the threshold
    Args:
        x: input tensor to apply the sparsemax
        batch: group indicators
    Returns:
        the threshold value
    """
    num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
    cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)
    sorted_input, input_cumsum = scatter_sort(x, batch)
    input_cumsum = input_cumsum - 1.0
    rhos = _make_ix_like(batch).to(x.dtype)
    support = rhos * sorted_input > input_cumsum
    support_size = scatter_add(support.to(batch.dtype), batch)
    idx = support_size + cum_num_nodes - 1
    mask = idx < 0
    idx[mask] = 0
    tau = input_cumsum.gather(0, idx)
    tau /= support_size.to(x.dtype)
    return tau, support_size
class SparsemaxFunction(Function):
    @staticmethod
    def forward(ctx, x, batch):
        """sparsemax: normalizing sparse transform
        Parameters:
            ctx: context object
            x (Tensor): shape (N, )
            batch: group indicator
        Returns:
            output (Tensor): same shape as input
        """
        max_val, _ = scatter_max(x, batch)
        x -= max_val[batch]
        tau, supp_size = _threshold_and_support(x, batch)
        output = torch.clamp(x - tau[batch], min=0)
        ctx.save_for_backward(supp_size, output, batch)
        return output
    @staticmethod
    def backward(ctx, grad_output):
        supp_size, output, batch = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[output == 0] = 0
        v_hat = scatter_add(grad_input, batch) / supp_size.to(output.dtype)
        grad_input = torch.where(output != 0, grad_input - v_hat[batch], grad_input)
        return grad_input, None
sparsemax = SparsemaxFunction.apply
class Sparsemax(nn.Module):
    def __init__(self):
        super(Sparsemax, self).__init__()
    def forward(self, x, batch):
        return sparsemax(x, batch)
if __name__ == '__main__':
    sparse_attention = Sparsemax()
    input_x = torch.tensor([1.7301, 0.6792, -1.0565, 1.6614, -0.3196, -0.7790, -0.3877, -0.4943, 0.1831, -0.0061])
    input_batch = torch.cat([torch.zeros(4, dtype=torch.long), torch.ones(6, dtype=torch.long)], dim=0)
    res = sparse_attention(input_x, input_batch)
    print(res)
class TwoHopNeighborhood(object):
    def __call__(self, data):
        edge_index, edge_attr = data.edge_index, data.edge_attr
        n = data.num_nodes
        fill = 1e16
        value = edge_index.new_full((edge_index.size(1),), fill, dtype=torch.float)
        index, value = spspmm(edge_index, value, edge_index, value, n, n, n, True)
        edge_index = torch.cat([edge_index, index], dim=1)
        if edge_attr is None:
            data.edge_index, _ = coalesce(edge_index, None, n, n)
        else:
            value = value.view(-1, *[1 for _ in range(edge_attr.dim() - 1)])
            value = value.expand(-1, *list(edge_attr.size())[1:])
            edge_attr = torch.cat([edge_attr, value], dim=0)
            data.edge_index, edge_attr = coalesce(edge_index, edge_attr, n, n, op='min', fill_value=fill)
            edge_attr[edge_attr >= fill] = 0
            data.edge_attr = edge_attr
        return data
    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)
class GCN(MessagePassing):
    def __init__(self, in_channels, out_channels, cached=False, bias=True, **kwargs):
        super(GCN, self).__init__(aggr='add', **kwargs)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None
        self.weight = Parameter(torch.Tensor(in_channels, out_channels))
        nn.init.xavier_uniform_(self.weight.data)
        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
            nn.init.zeros_(self.bias.data)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
    def reset_parameters(self):
        self.cached_result = None
        self.cached_num_edges = None
    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
    def forward(self, x, edge_index, edge_weight=None):
        x = torch.matmul(x, self.weight)
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))
        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm
        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out
    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels, self.out_channels)
class NodeInformationScore(MessagePassing):
    def __init__(self, improved=False, cached=False, **kwargs):
        super(NodeInformationScore, self).__init__(aggr='add', **kwargs)
        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None
    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        edge_index, edge_weight = add_remaining_self_loops(edge_index, edge_weight, 0, num_nodes)
        row, col = edge_index
        expand_deg = torch.zeros((edge_weight.size(0),), dtype=dtype, device=edge_index.device)
        expand_deg[-num_nodes:] = torch.ones((num_nodes,), dtype=dtype, device=edge_index.device)
        return edge_index, expand_deg - deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
    def forward(self, x, edge_index, edge_weight):
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))
        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm
        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    def update(self, aggr_out):
        return aggr_out
class HGPSLPool(torch.nn.Module):
    def __init__(self, in_channels, ratio=0.5, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2):
        super(HGPSLPool, self).__init__()
        self.in_channels = in_channels
        self.ratio = ratio
        self.sample = sample
        self.sparse = sparse
        self.sl = sl
        self.negative_slop = negative_slop
        self.lamb = lamb
        self.att = Parameter(torch.Tensor(1, self.in_channels * 2))
        nn.init.xavier_uniform_(self.att.data)
        self.sparse_attention = Sparsemax()
        self.neighbor_augment = TwoHopNeighborhood()
        self.calc_information_score = NodeInformationScore()
    def forward(self, x, edge_index, edge_attr, batch):
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        x_information_score = self.calc_information_score(x, edge_index, edge_attr)
        score = torch.sum(torch.abs(x_information_score), dim=1)
        original_x = x
        perm = topk(score, self.ratio, batch)
        x = x[perm]
        batch = batch[perm]
        induced_edge_index, induced_edge_attr = filter_adj(edge_index, edge_attr, perm, num_nodes=score.size(0))
        if self.sl is False:
            return x, induced_edge_index, induced_edge_attr, batch
        if self.sample:
            k_hop = 3
            if edge_attr is None:
                edge_attr = torch.ones((edge_index.size(1),), dtype=torch.float, device=edge_index.device)
            hop_data = Data(x=original_x, edge_index=edge_index, edge_attr=edge_attr)
            for _ in range(k_hop - 1):
                hop_data = self.neighbor_augment(hop_data)
            hop_edge_index = hop_data.edge_index
            hop_edge_attr = hop_data.edge_attr
            new_edge_index, new_edge_attr = filter_adj(hop_edge_index, hop_edge_attr, perm, num_nodes=score.size(0))
            new_edge_index, new_edge_attr = add_remaining_self_loops(new_edge_index, new_edge_attr, 0, x.size(0))
            row, col = new_edge_index
            weights = (torch.cat([x[row], x[col]], dim=1) * self.att).sum(dim=-1)
            weights = F.leaky_relu(weights, self.negative_slop) + new_edge_attr * self.lamb
            adj = torch.zeros((x.size(0), x.size(0)), dtype=torch.float, device=x.device)
            adj[row, col] = weights
            new_edge_index, weights = dense_to_sparse(adj)
            row, col = new_edge_index
            if self.sparse:
                new_edge_attr = self.sparse_attention(weights, row)
            else:
                new_edge_attr = softmax(weights, row, x.size(0))
            adj[row, col] = new_edge_attr
            new_edge_index, new_edge_attr = dense_to_sparse(adj)
            del adj
            torch.cuda.empty_cache()
        else:
            if edge_attr is None:
                induced_edge_attr = torch.ones((induced_edge_index.size(1),), dtype=x.dtype,
                                               device=induced_edge_index.device)
            num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
            shift_cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)
            cum_num_nodes = num_nodes.cumsum(dim=0)
            adj = torch.zeros((x.size(0), x.size(0)), dtype=torch.float, device=x.device)
            for idx_i, idx_j in zip(shift_cum_num_nodes, cum_num_nodes):
                adj[idx_i:idx_j, idx_i:idx_j] = 1.0
            new_edge_index, _ = dense_to_sparse(adj)
            row, col = new_edge_index
            weights = (torch.cat([x[row], x[col]], dim=1) * self.att).sum(dim=-1)
            weights = F.leaky_relu(weights, self.negative_slop)
            adj[row, col] = weights
            induced_row, induced_col = induced_edge_index
            adj[induced_row, induced_col] += induced_edge_attr * self.lamb
            weights = adj[row, col]
            if self.sparse:
                new_edge_attr = self.sparse_attention(weights, row)
            else:
                new_edge_attr = softmax(weights, row, num_nodes=x.size(0))
            adj[row, col] = new_edge_attr
            new_edge_index, new_edge_attr = dense_to_sparse(adj)
            del adj
            torch.cuda.empty_cache()
        return x, new_edge_index, new_edge_attr, batch

tensor([0.5344, 0.0000, 0.0000, 0.4656, 0.0613, 0.0000, 0.0000, 0.0000, 0.5640,
        0.3748])


### MUTAG

In [6]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 150
data_path = "/data1/Pooling/1"
dataset_sparse = TUDataset(root=data_path, name="MUTAG", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class HierarchicalGCN_HGPSL(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_HGPSL, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = HGPSLPool(hidden_channels, ratio=0.9, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = HGPSLPool(hidden_channels, ratio=0.9, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        edge_attr=None
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool1(x, edge_index, edge_attr, batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool2(x, edge_index, edge_attr, batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)
    model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Early stopping at epoch 165 for seed 43
Early stopping at epoch 152 for seed 44
Average Time: 11.07 seconds
Var Time: 2.06 seconds
Average Memory: 24.00 MB
Average Best Val Acc: 0.7976
Std Best Test Acc: 0.1270
Average Test Acc: 0.7126


### DD

In [10]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data1/Pooling"
dataset_sparse = TUDataset(root=data_path, name="DD", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class HierarchicalGCN_HGPSL(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_HGPSL, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = HGPSLPool(hidden_channels, ratio=0.5, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = HGPSLPool(hidden_channels, ratio=0.5, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        edge_attr = None
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool1(x, edge_index, edge_attr, batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool2(x, edge_index, edge_attr, batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)
    model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Average Time: 122.73 seconds
Var Time: 0.90 seconds
Average Memory: 10767.33 MB
Average Best Val Acc: 0.7748
Std Best Test Acc: 0.0278
Average Test Acc: 0.7327


In [7]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data1/Pooling"
dataset_sparse = TUDataset(root=data_path, name="IMDB-BINARY", transform=T.Compose([T.OneHotDegree(136)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class HierarchicalGCN_HGPSL(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_HGPSL, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = HGPSLPool(hidden_channels, ratio=0.9, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = HGPSLPool(hidden_channels, ratio=0.9, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        edge_attr = None
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool1(x, edge_index, edge_attr, batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool2(x, edge_index, edge_attr, batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)
    model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6963, Val Acc: 0.4800, Test Acc: 0.4733
Seed: 42, Epoch: 002, Loss: 0.6957, Val Acc: 0.4800, Test Acc: 0.4733
Seed: 42, Epoch: 003, Loss: 0.6953, Val Acc: 0.4800, Test Acc: 0.4733
Seed: 42, Epoch: 004, Loss: 0.6949, Val Acc: 0.4800, Test Acc: 0.4733
Seed: 42, Epoch: 005, Loss: 0.6943, Val Acc: 0.4800, Test Acc: 0.4733
Seed: 42, Epoch: 006, Loss: 0.6937, Val Acc: 0.4800, Test Acc: 0.4733
Seed: 42, Epoch: 007, Loss: 0.6933, Val Acc: 0.4800, Test Acc: 0.4733
Seed: 42, Epoch: 008, Loss: 0.6927, Val Acc: 0.4800, Test Acc: 0.4733
Seed: 42, Epoch: 009, Loss: 0.6918, Val Acc: 0.4867, Test Acc: 0.4733
Seed: 42, Epoch: 010, Loss: 0.6907, Val Acc: 0.5267, Test Acc: 0.4733
Seed: 42, Epoch: 011, Loss: 0.6908, Val Acc: 0.5467, Test Acc: 0.5400
Seed: 42, Epoch: 012, Loss: 0.6889, Val Acc: 0.5533, Test Acc: 0.5600
Seed: 42, Epoch: 013, Loss: 0.6876, Val Acc: 0.6267, Test Acc: 0.6000
Seed: 42, Epoch: 014, Loss: 0.6852, Val Acc: 0.7267, Test Acc: 0.6000
Seed: 42, Epoch: 015

### IMDB-MULTI

In [12]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data1/Pooling"
dataset_sparse = TUDataset(root=data_path, name="IMDB-MULTI", transform=T.Compose([T.OneHotDegree(88)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class HierarchicalGCN_HGPSL(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_HGPSL, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = HGPSLPool(hidden_channels, ratio=0.7, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = HGPSLPool(hidden_channels, ratio=0.7, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        edge_attr = None
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool1(x, edge_index, edge_attr, batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool2(x, edge_index, edge_attr, batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)
    model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 1.1004, Val Acc: 0.3378, Test Acc: 0.3822
Seed: 42, Epoch: 002, Loss: 1.0999, Val Acc: 0.2933, Test Acc: 0.3867
Seed: 42, Epoch: 003, Loss: 1.0993, Val Acc: 0.2800, Test Acc: 0.3778
Seed: 42, Epoch: 004, Loss: 1.0991, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 005, Loss: 1.0985, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 006, Loss: 1.0982, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 007, Loss: 1.0979, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 008, Loss: 1.0976, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 009, Loss: 1.0970, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 010, Loss: 1.0967, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 011, Loss: 1.0962, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 012, Loss: 1.0963, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 013, Loss: 1.0955, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 014, Loss: 1.0944, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 015

### COLLAB

In [13]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
data_path = "/data1/Pooling"
dataset_sparse = TUDataset(root=data_path, name="COLLAB", transform=T.Compose([T.OneHotDegree(491)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class HierarchicalGCN_HGPSL(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_HGPSL, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = HGPSLPool(hidden_channels, ratio=0.9, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = HGPSLPool(hidden_channels, ratio=0.9, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        edge_attr = None
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool1(x, edge_index, edge_attr, batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch = self.pool2(x, edge_index, edge_attr, batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)
    model = HierarchicalGCN_HGPSL(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 1.1104, Val Acc: 0.3213, Test Acc: 0.3320
Seed: 42, Epoch: 002, Loss: 1.1003, Val Acc: 0.3213, Test Acc: 0.3320
Seed: 42, Epoch: 003, Loss: 1.0890, Val Acc: 0.3213, Test Acc: 0.3320
Seed: 42, Epoch: 004, Loss: 1.0746, Val Acc: 0.5880, Test Acc: 0.5800
Seed: 42, Epoch: 005, Loss: 1.0556, Val Acc: 0.6080, Test Acc: 0.6227
Seed: 42, Epoch: 006, Loss: 1.0261, Val Acc: 0.6173, Test Acc: 0.6280
Seed: 42, Epoch: 007, Loss: 0.9842, Val Acc: 0.6133, Test Acc: 0.6280
Seed: 42, Epoch: 008, Loss: 0.9317, Val Acc: 0.6173, Test Acc: 0.6280
Seed: 42, Epoch: 009, Loss: 0.8763, Val Acc: 0.6187, Test Acc: 0.6280
Seed: 42, Epoch: 010, Loss: 0.8179, Val Acc: 0.6320, Test Acc: 0.6533
Seed: 42, Epoch: 011, Loss: 0.7563, Val Acc: 0.6493, Test Acc: 0.6587
Seed: 42, Epoch: 012, Loss: 0.7014, Val Acc: 0.6653, Test Acc: 0.6653
Seed: 42, Epoch: 013, Loss: 0.6582, Val Acc: 0.6707, Test Acc: 0.6693
Seed: 42, Epoch: 014, Loss: 0.6282, Val Acc: 0.6787, Test Acc: 0.6773
Seed: 42, Epoch: 015

# Graph Regression

### QM7

In [8]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --run_times=5 --patience=150 --epochs=500 --hgpsl_ratio=0.1 --pooling='HGPSL'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --run_times=5 --patience=150 --epochs=500 --hgpsl_ratio=0.3 --pooling='HGPSL'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --run_times=5 --patience=150 --epochs=500 --hgpsl_ratio=0.5 --pooling='HGPSL'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --run_times=5 --patience=150 --epochs=500 --hgpsl_ratio=0.7 --pooling='HGPSL'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --run_times=5 --patience=150 --epochs=500 --hgpsl_ratio=0.9 --pooling='HGPSL'

++++++++++++++++++++++0.1++++++++++++++++++++++++
Generating dataset...
Packaging molecules, finish 100.0%

Current dataset: qm7, include 6832 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/500MAE=1549.8635 MAE=1547.9595 MAE=1545.5062 MAE=1543.0140 MAE=1540.4630 MAE=1537.4766 MAE=1534.4363 MAE=1530.2174 MAE=1526.0850 Epoch: 10/500MAE=1521.9048 MAE=1517.2203 MAE=1511.4885 MAE=1506.5045 MAE=1501.0889 MAE=1494.7964 MAE=1488.3619 MAE=1481.3239 MAE=1473.1382 MAE=1465.1357 Epoch: 20/500MAE=1457.5217 MAE=1448.8146 MAE=1439.3210 MAE=1430.4114 MAE=1421.4143 MAE=1411.2148 MAE=1399.9261 MAE=1390.0613 MAE=1379.6327 MAE=1367.7371 Epoch: 30/500MAE=1357.7585 MAE=1345.2727 MAE=1334.4143 MAE=1322.5632 MAE=1307.4254 MAE=1294.1014 MAE=1282.1926 MAE=1269.1797 MAE=1258.5542 MAE=1240.6697 Epoch: 40/500MAE=1222.6736 MAE=1205.5114 MAE=1181.1680 MAE=1157.3481 MAE=1142.2969 MAE=1125.6968 MAE=1104.4193 MAE=1088.2207 MAE=1069.4326 MAE=1048.0422 Epoch: 50/500MAE=1029.2899 MAE=1009.1296 MAE=989.

### QM8

In [6]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --run_times=5 --patience=10 --epochs=150 --hgpsl_ratio=0.1 --pooling='HGPSL'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --run_times=5 --patience=10 --epochs=150 --hgpsl_ratio=0.3 --pooling='HGPSL'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --run_times=5 --patience=10 --epochs=150 --hgpsl_ratio=0.5 --pooling='HGPSL'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --run_times=5 --patience=10 --epochs=150 --hgpsl_ratio=0.7 --pooling='HGPSL'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --run_times=5 --patience=10 --epochs=150 --hgpsl_ratio=0.9 --pooling='HGPSL'

++++++++++++++++++++++0.1++++++++++++++++++++++++
Generating dataset...
Packaging molecules, finish 100.0%

Current dataset: qm8, include 21783 molecules and 12 regression tasks

Splitting, finish 1/1  
Epoch: 1/150MAE=0.1300 MAE=0.1249 MAE=0.1030 MAE=0.0756 MAE=0.0593 MAE=0.0502 MAE=0.0453 MAE=0.0426 MAE=0.0376 Epoch: 10/150MAE=0.0353 MAE=0.0332 MAE=0.0318 MAE=0.0317 MAE=0.0302 MAE=0.0287 MAE=0.0288 MAE=0.0278 MAE=0.0283 MAE=0.0278 Epoch: 20/150MAE=0.0279 MAE=0.0270 MAE=0.0270 MAE=0.0267 MAE=0.0264 MAE=0.0262 MAE=0.0262 MAE=0.0264 MAE=0.0264 MAE=0.0269 Epoch: 30/150MAE=0.0260 MAE=0.0259 MAE=0.0259 MAE=0.0264 MAE=0.0259 MAE=0.0257 MAE=0.0260 MAE=0.0258 MAE=0.0259 MAE=0.0258 Epoch: 40/150MAE=0.0252 MAE=0.0253 MAE=0.0251 MAE=0.0253 MAE=0.0250 MAE=0.0250 MAE=0.0250 MAE=0.0248 MAE=0.0251 MAE=0.0248 Epoch: 50/150MAE=0.0249 MAE=0.0251 MAE=0.0246 MAE=0.0247 MAE=0.0245 MAE=0.0245 MAE=0.0246 MAE=0.0245 MAE=0.0246 MAE=0.0246 Epoch: 60/150MAE=0.0247 MAE=0.0245 MAE=0.0245 MAE=0.0245 MAE=0.0244 MAE

### BACE

In [5]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.1 --pooling='HGPSL'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.3 --pooling='HGPSL'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.5 --pooling='HGPSL'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.7 --pooling='HGPSL'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.9 --pooling='HGPSL'

++++++++++++++++++++++0.1++++++++++++++++++++++++


Generating dataset...
Packaging molecules, finish 100.0%

Current dataset: bace, include 1513 molecules and 1 classification tasks

Splitting, finish 1/1  
Epoch: 1/150Epoch: 10/150Epoch: 20/150Epoch: 30/150
********************1's fold 1's run over********************
AUROC: 0.839 +/- 0.000
AUPRC: 0.796 +/- 0.000

Epoch: 1/150Epoch: 10/150Epoch: 20/150Epoch: 30/150
********************1's fold 2's run over********************
AUROC: 0.830 +/- 0.009
AUPRC: 0.796 +/- 0.000

Epoch: 1/150Epoch: 10/150Epoch: 20/150
********************1's fold 3's run over********************
AUROC: 0.829 +/- 0.008
AUPRC: 0.791 +/- 0.007

Epoch: 1/150Epoch: 10/150Epoch: 20/150Epoch: 30/150
********************1's fold 4's run over********************
AUROC: 0.838 +/- 0.017
AUPRC: 0.801 +/- 0.019

Epoch: 1/150Epoch: 10/150Epoch: 20/150Epoch: 30/150
********************1's fold 5's run over********************
AUROC: 0.841 +/- 0.016
AUPRC: 0.801 +/- 0.017

++++++++++++++++++++++0.3++++++++++++++++++++++++
Ge

### ESOL

In [1]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python -W ignore /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.1 --pooling='HGPSL'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python -W ignore /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.3 --pooling='HGPSL'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python -W ignore /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.5 --pooling='HGPSL'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python -W ignore /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.7 --pooling='HGPSL'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python -W ignore /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --cuda_num 0 --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.9 --pooling='HGPSL'

++++++++++++++++++++++0.1++++++++++++++++++++++++
Generating dataset...
Packaging molecules, finish 100.0%

Current dataset: esol, include 1127 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/150RMSE=1.5778 RMSE=1.4261 RMSE=1.2931 RMSE=1.3573 RMSE=1.3921 RMSE=1.3780 RMSE=1.2969 RMSE=1.3914 RMSE=1.3335 Epoch: 10/150RMSE=1.3878 RMSE=1.3276 RMSE=1.3144 RMSE=1.3332 RMSE=1.3484 RMSE=1.3725 RMSE=1.3852 RMSE=1.3747 RMSE=1.4050 RMSE=1.4163 Epoch: 20/150RMSE=1.4126 RMSE=1.4114 RMSE=1.4043 RMSE=1.3772 RMSE=1.4000 RMSE=1.4171 RMSE=1.4196 RMSE=1.4236 RMSE=1.6002 
********************1's fold 1's run over********************
RMSE: 1.600 +/- 0.000

Epoch: 1/150RMSE=1.6612 RMSE=1.3769 RMSE=1.4377 RMSE=1.3524 RMSE=1.3245 RMSE=1.4048 RMSE=1.4792 RMSE=1.3655 RMSE=1.4407 Epoch: 10/150RMSE=1.3751 RMSE=1.3638 RMSE=1.4074 RMSE=1.4074 RMSE=1.3561 RMSE=1.3588 RMSE=1.3978 RMSE=1.3469 RMSE=1.3454 RMSE=1.3460 Epoch: 20/150RMSE=1.3340 RMSE=1.3449 RMSE=1.3433 RMSE=1.3471 RMSE=1.3549 RMSE=1.3474 

### Freesolv

In [2]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python -W ignore  /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.1 --pooling='HGPSL'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python -W ignore  /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.3 --pooling='HGPSL'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python -W ignore  /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.5 --pooling='HGPSL'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python -W ignore  /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.7 --pooling='HGPSL'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python -W ignore  /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.9 --pooling='HGPSL'

++++++++++++++++++++++0.1++++++++++++++++++++++++
Generating dataset...
Packaging molecules, finish 100.0%

Current dataset: freesolv, include 639 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/150RMSE=5.2482 RMSE=4.5708 RMSE=3.2658 RMSE=2.2828 RMSE=2.1757 RMSE=2.1363 RMSE=2.0819 RMSE=2.1122 RMSE=2.2811 Epoch: 10/150RMSE=2.0639 RMSE=2.1916 RMSE=2.2379 RMSE=2.4131 RMSE=1.9504 RMSE=1.8215 RMSE=2.0446 RMSE=2.0446 RMSE=2.0372 RMSE=1.9190 Epoch: 20/150RMSE=1.9083 RMSE=1.8567 RMSE=1.9782 RMSE=1.9589 RMSE=2.0076 RMSE=1.9407 RMSE=1.9879 RMSE=1.9908 RMSE=1.9344 RMSE=1.8616 Epoch: 30/150RMSE=1.8475 RMSE=1.9225 RMSE=1.8375 RMSE=1.8373 RMSE=1.9416 RMSE=1.8420 RMSE=3.0029 
********************1's fold 1's run over********************
RMSE: 3.003 +/- 0.000

Epoch: 1/150RMSE=5.3558 RMSE=4.7592 RMSE=3.6294 RMSE=2.6478 RMSE=2.3354 RMSE=2.1562 RMSE=2.2082 RMSE=2.0188 RMSE=1.9637 Epoch: 10/150RMSE=1.9438 RMSE=2.0948 RMSE=1.8186 RMSE=2.0114 RMSE=1.9046 RMSE=1.8911 RMSE=2.1982 RMSE=1.75

### Lipophilicity

In [3]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.1 --pooling='HGPSL'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.3 --pooling='HGPSL'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.5 --pooling='HGPSL'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.7 --pooling='HGPSL'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --hgpsl_ratio=0.9 --pooling='HGPSL'

++++++++++++++++++++++0.1++++++++++++++++++++++++
Generating dataset...
Packaging molecules, finish 100.0%

Current dataset: lipo, include 4200 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/150RMSE=1.2876 RMSE=1.1484 RMSE=1.1440 RMSE=1.1290 RMSE=1.1224 RMSE=1.1190 RMSE=1.1058 RMSE=1.1097 RMSE=1.1513 Epoch: 10/150RMSE=1.1016 RMSE=1.0863 RMSE=1.1012 RMSE=1.0855 RMSE=1.0780 RMSE=1.0973 RMSE=1.0584 RMSE=1.0896 RMSE=1.0731 RMSE=1.0758 Epoch: 20/150RMSE=1.0860 RMSE=1.0534 RMSE=1.0609 RMSE=1.0833 RMSE=1.0769 RMSE=1.0836 RMSE=1.0602 RMSE=1.0572 RMSE=1.0550 RMSE=1.0669 Epoch: 30/150RMSE=1.0507 RMSE=1.0484 RMSE=1.0496 RMSE=1.0571 RMSE=1.0650 RMSE=1.0663 RMSE=1.0630 RMSE=1.0674 RMSE=1.0646 RMSE=1.0753 Epoch: 40/150RMSE=1.0727 RMSE=1.0721 RMSE=1.0792 RMSE=1.0712 RMSE=1.0725 RMSE=1.0706 RMSE=1.0607 RMSE=1.0692 RMSE=1.0638 RMSE=1.0654 Epoch: 50/150RMSE=1.0717 RMSE=1.0683 RMSE=0.9785 
********************1's fold 1's run over********************
RMSE: 0.979 +/- 0.000

Epoch: 1/15

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Function
from torch_scatter import scatter_add, scatter_max
from torch_geometric.utils import softmax, dense_to_sparse, add_remaining_self_loops
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from dataclasses import dataclass
from typing import Optional
import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
import torch
import torch.nn as nn
import numpy as np
from typing import Union, Optional, Callable
from torch_scatter import scatter_add, scatter_max
from torch_geometric.utils import softmax
import math
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GCNConv, GATConv, ChebConv, GraphConv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.data import Data
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import softmax, dense_to_sparse, add_remaining_self_loops
from torch_scatter import scatter_add
from torch_sparse import spspmm, coalesce
import torch
import torch.nn as nn
from torch.autograd import Function
from torch_scatter import scatter_add, scatter_max
def topk(x, ratio, batch, min_score=None, tol=1e-7):
    if min_score is not None:
        scores_max = scatter_max(x, batch)[0][batch] - tol
        scores_min = scores_max.clamp(max=min_score)
        perm = torch.nonzero(x > scores_min).view(-1)
    else:
        num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
        batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()
        cum_num_nodes = torch.cat(
            [num_nodes.new_zeros(1),
            num_nodes.cumsum(dim=0)[:-1]], dim=0)
        index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
        index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
        dense_x = x.new_full((batch_size * max_num_nodes, ), -2)
        dense_x[index] = x
        dense_x = dense_x.view(batch_size, max_num_nodes)
        _, perm = dense_x.sort(dim=-1, descending=True)
        perm = perm + cum_num_nodes.view(-1, 1)
        perm = perm.view(-1)
        k = (ratio * num_nodes.to(torch.float)).ceil().to(torch.long)
        mask = [
            torch.arange(k[i], dtype=torch.long, device=x.device) +
            i * max_num_nodes for i in range(batch_size)
        ]
        mask = torch.cat(mask, dim=0)
        perm = perm[mask]
    return perm
def filter_adj(edge_index, edge_weight, perm, num_nodes=None):
        num_nodes = maybe_num_nodes(edge_index, num_nodes)
        mask = perm.new_full((num_nodes, ), -1)
        i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
        mask[perm] = i
        row, col = edge_index
        row, col = mask[row], mask[col]
        mask = (row >= 0) & (col >= 0)
        row, col = row[mask], col[mask]
        if edge_weight is not None:
            edge_weight = edge_weight[mask]
        return torch.stack([row, col], dim=0), edge_weight
def scatter_sort(x, batch, fill_value=-1e16):
    num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
    batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()
    cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)
    index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
    index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)
    dense_x = x.new_full((batch_size * max_num_nodes,), fill_value)
    dense_x[index] = x
    dense_x = dense_x.view(batch_size, max_num_nodes)
    sorted_x, _ = dense_x.sort(dim=-1, descending=True)
    cumsum_sorted_x = sorted_x.cumsum(dim=-1)
    cumsum_sorted_x = cumsum_sorted_x.view(-1)
    sorted_x = sorted_x.view(-1)
    filled_index = sorted_x != fill_value
    sorted_x = sorted_x[filled_index]
    cumsum_sorted_x = cumsum_sorted_x[filled_index]
    return sorted_x, cumsum_sorted_x
def _make_ix_like(batch):
    num_nodes = scatter_add(batch.new_ones(batch.size(0)), batch, dim=0)
    idx = [torch.arange(1, i + 1, dtype=torch.long, device=batch.device) for i in num_nodes]
    idx = torch.cat(idx, dim=0)
    return idx
def _threshold_and_support(x, batch):
    """Sparsemax building block: compute the threshold
    Args:
        x: input tensor to apply the sparsemax
        batch: group indicators
    Returns:
        the threshold value
    """
    num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
    cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)
    sorted_input, input_cumsum = scatter_sort(x, batch)
    input_cumsum = input_cumsum - 1.0
    rhos = _make_ix_like(batch).to(x.dtype)
    support = rhos * sorted_input > input_cumsum
    support_size = scatter_add(support.to(batch.dtype), batch)
    idx = support_size + cum_num_nodes - 1
    mask = idx < 0
    idx[mask] = 0
    tau = input_cumsum.gather(0, idx)
    tau /= support_size.to(x.dtype)
    return tau, support_size
class SparsemaxFunction(Function):
    @staticmethod
    def forward(ctx, x, batch):
        """sparsemax: normalizing sparse transform
        Parameters:
            ctx: context object
            x (Tensor): shape (N, )
            batch: group indicator
        Returns:
            output (Tensor): same shape as input
        """
        max_val, _ = scatter_max(x, batch)
        x -= max_val[batch]
        tau, supp_size = _threshold_and_support(x, batch)
        output = torch.clamp(x - tau[batch], min=0)
        ctx.save_for_backward(supp_size, output, batch)
        return output
    @staticmethod
    def backward(ctx, grad_output):
        supp_size, output, batch = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[output == 0] = 0
        v_hat = scatter_add(grad_input, batch) / supp_size.to(output.dtype)
        grad_input = torch.where(output != 0, grad_input - v_hat[batch], grad_input)
        return grad_input, None
sparsemax = SparsemaxFunction.apply
class Sparsemax(nn.Module):
    def __init__(self):
        super(Sparsemax, self).__init__()
    def forward(self, x, batch):
        return sparsemax(x, batch)
if __name__ == '__main__':
    sparse_attention = Sparsemax()
    input_x = torch.tensor([1.7301, 0.6792, -1.0565, 1.6614, -0.3196, -0.7790, -0.3877, -0.4943, 0.1831, -0.0061])
    input_batch = torch.cat([torch.zeros(4, dtype=torch.long), torch.ones(6, dtype=torch.long)], dim=0)
    res = sparse_attention(input_x, input_batch)
    print(res)
class TwoHopNeighborhood(object):
    def __call__(self, data):
        edge_index, edge_attr = data.edge_index, data.edge_attr
        n = data.num_nodes
        fill = 1e16
        value = edge_index.new_full((edge_index.size(1),), fill, dtype=torch.float)
        index, value = spspmm(edge_index, value, edge_index, value, n, n, n, True)
        edge_index = torch.cat([edge_index, index], dim=1)
        if edge_attr is None:
            data.edge_index, _ = coalesce(edge_index, None, n, n)
        else:
            value = value.view(-1, *[1 for _ in range(edge_attr.dim() - 1)])
            value = value.expand(-1, *list(edge_attr.size())[1:])
            edge_attr = torch.cat([edge_attr, value], dim=0)
            data.edge_index, edge_attr = coalesce(edge_index, edge_attr, n, n, op='min', fill_value=fill)
            edge_attr[edge_attr >= fill] = 0
            data.edge_attr = edge_attr
        return data
    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)
class GCN(MessagePassing):
    def __init__(self, in_channels, out_channels, cached=False, bias=True, **kwargs):
        super(GCN, self).__init__(aggr='add', **kwargs)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None
        self.weight = Parameter(torch.Tensor(in_channels, out_channels))
        nn.init.xavier_uniform_(self.weight.data)
        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
            nn.init.zeros_(self.bias.data)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
    def reset_parameters(self):
        self.cached_result = None
        self.cached_num_edges = None
    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
    def forward(self, x, edge_index, edge_weight=None):
        x = torch.matmul(x, self.weight)
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))
        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm
        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out
    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels, self.out_channels)
class NodeInformationScore(MessagePassing):
    def __init__(self, improved=False, cached=False, **kwargs):
        super(NodeInformationScore, self).__init__(aggr='add', **kwargs)
        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None
    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        edge_index, edge_weight = add_remaining_self_loops(edge_index, edge_weight, 0, num_nodes)
        row, col = edge_index
        expand_deg = torch.zeros((edge_weight.size(0),), dtype=dtype, device=edge_index.device)
        expand_deg[-num_nodes:] = torch.ones((num_nodes,), dtype=dtype, device=edge_index.device)
        return edge_index, expand_deg - deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
    def forward(self, x, edge_index, edge_weight):
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))
        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm
        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    def update(self, aggr_out):
        return aggr_out
class HGPSLPool(torch.nn.Module):
    def __init__(self, in_channels, ratio=0.5, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2):
        super(HGPSLPool, self).__init__()
        self.in_channels = in_channels
        self.ratio = ratio
        self.sample = sample
        self.sparse = sparse
        self.sl = sl
        self.negative_slop = negative_slop
        self.lamb = lamb
        self.att = Parameter(torch.Tensor(1, self.in_channels * 2))
        nn.init.xavier_uniform_(self.att.data)
        self.sparse_attention = Sparsemax()
        self.neighbor_augment = TwoHopNeighborhood()
        self.calc_information_score = NodeInformationScore()
    def forward(self, x, edge_index, edge_attr, batch):
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        x_information_score = self.calc_information_score(x, edge_index, edge_attr)
        score = torch.sum(torch.abs(x_information_score), dim=1)
        original_x = x
        perm = topk(score, self.ratio, batch)
        x = x[perm]
        batch = batch[perm]
        induced_edge_index, induced_edge_attr = filter_adj(edge_index, edge_attr, perm, num_nodes=score.size(0))
        if self.sl is False:
            return x, induced_edge_index, induced_edge_attr, batch
        if self.sample:
            k_hop = 3
            if edge_attr is None:
                edge_attr = torch.ones((edge_index.size(1),), dtype=torch.float, device=edge_index.device)
            hop_data = Data(x=original_x, edge_index=edge_index, edge_attr=edge_attr)
            for _ in range(k_hop - 1):
                hop_data = self.neighbor_augment(hop_data)
            hop_edge_index = hop_data.edge_index
            hop_edge_attr = hop_data.edge_attr
            new_edge_index, new_edge_attr = filter_adj(hop_edge_index, hop_edge_attr, perm, num_nodes=score.size(0))
            new_edge_index, new_edge_attr = add_remaining_self_loops(new_edge_index, new_edge_attr, 0, x.size(0))
            row, col = new_edge_index
            weights = (torch.cat([x[row], x[col]], dim=1) * self.att).sum(dim=-1)
            weights = F.leaky_relu(weights, self.negative_slop) + new_edge_attr * self.lamb
            adj = torch.zeros((x.size(0), x.size(0)), dtype=torch.float, device=x.device)
            adj[row, col] = weights
            new_edge_index, weights = dense_to_sparse(adj)
            row, col = new_edge_index
            if self.sparse:
                new_edge_attr = self.sparse_attention(weights, row)
            else:
                new_edge_attr = softmax(weights, row, x.size(0))
            adj[row, col] = new_edge_attr
            new_edge_index, new_edge_attr = dense_to_sparse(adj)
            del adj
            torch.cuda.empty_cache()
        else:
            if edge_attr is None:
                induced_edge_attr = torch.ones((induced_edge_index.size(1),), dtype=x.dtype,
                                               device=induced_edge_index.device)
            num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
            shift_cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)
            cum_num_nodes = num_nodes.cumsum(dim=0)
            adj = torch.zeros((x.size(0), x.size(0)), dtype=torch.float, device=x.device)
            for idx_i, idx_j in zip(shift_cum_num_nodes, cum_num_nodes):
                adj[idx_i:idx_j, idx_i:idx_j] = 1.0
            new_edge_index, _ = dense_to_sparse(adj)
            row, col = new_edge_index
            weights = (torch.cat([x[row], x[col]], dim=1) * self.att).sum(dim=-1)
            weights = F.leaky_relu(weights, self.negative_slop)
            adj[row, col] = weights
            induced_row, induced_col = induced_edge_index
            adj[induced_row, induced_col] += induced_edge_attr * self.lamb
            weights = adj[row, col]
            if self.sparse:
                new_edge_attr = self.sparse_attention(weights, row)
            else:
                new_edge_attr = softmax(weights, row, num_nodes=x.size(0))
            adj[row, col] = new_edge_attr
            new_edge_index, new_edge_attr = dense_to_sparse(adj)
            del adj
            torch.cuda.empty_cache()
        return x, new_edge_index, new_edge_attr, batch, perm

tensor([0.5344, 0.0000, 0.0000, 0.4656, 0.0613, 0.0000, 0.0000, 0.0000, 0.5640,
        0.3748])


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import random
import pandas as pd
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import CitationFull
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import shutil
dataset_sparse = Planetoid(root="/data1/Pooling", name='PubMed')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


In [4]:
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 1
pool_ratios = [0.5, 0.5]  
class HierarchicalGCN_HGPSL(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_HGPSL, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res
        channels = self.hidden_channels
        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(HGPSLPool(hidden_channels, ratio=0.5, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2))
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        edge_attr = None
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)
        xs = [x]
        edge_indices = [edge_index]
        perms = []
        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, perm = self.pools[i - 1](x, edge_index, edge_attr, batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]
            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)
        return x
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()
        val_acc = eval_node_classifier(model, graph, val_mask)
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  
        else:
            patience_counter += 1  
        if patience_counter >= patience:
            break
    return model, best_val_acc
def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc
kf = KFold(n_splits=5, shuffle=True)
seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        model = HierarchicalGCN_HGPSL(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        train_mask[train_index] = True
        test_mask[test_index] = True
        val_mask = test_mask  
        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)
        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')
    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  
    else:
        gpu_memory_usage = 0
    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')
for result in results:
    print(result)
mean_val_acc_values = [result['mean_val_acc'] for result in results]
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)
print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

RuntimeError: CUDA out of memory. Tried to allocate 46.35 GiB (GPU 0; 79.10 GiB total capacity; 48.61 GiB already allocated; 17.88 GiB free; 48.62 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF